# 1. Download SP500 Price Data

## 1.1 Import all necessary libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplt
import statsmodels.api as sm
import pandas_datareader.data as web
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
from statsmodels.regression.rolling import RollingOLS

In [9]:
warnings.filterwarnings('ignore')

## 1.2 Download the SP500 Constituent Data

In [10]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
print(type(sp500[0]))


<class 'pandas.core.frame.DataFrame'>


## 1.3 Clean the Data

- Isolate the ticker symbols
- Store the isoalted ticker symbols into a list

In [11]:
tickers_list = sp500[0]["Symbol"].tolist()

## 1.4 Take the start and end date to determine a range

- For this example, I will use 8 years

In [23]:
end_date = dt.datetime.today()
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

## 1.5.1 Per ticker, download the necessary information from the start date to the end date

In [24]:
df = yf.download(tickers=tickers_list, 
                 start = start_date, 
                 end = end_date).stack()
df

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2016-05-05 16:08:34.621139 -> 2024-05-03 16:08:34.621139)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2016-05-05 A        38.436123   40.959999   41.029999   40.509998   40.639999   
           AAL      31.531179   32.810001   33.520000   32.730000   33.279999   
           AAPL     21.438957   23.309999   23.517500   23.170000   23.500000   
           ABBV     44.179379   62.340000   62.400002   61.090000   61.410000   
           ABT      33.229504   38.500000   38.730000   38.099998   38.279999   
...                       ...         ...         ...         ...         ...   
2024-05-02 XYL     135.990005  135.990005  137.179993  132.020004  132.429993   
           YUM     135.070007  135.070007  136.610001  134.759995  135.500000   
           ZBH     118.459999  118.459999  123.959999  117.599998  121.989998   
           ZBRA    312.709991  312.709991  315.170013  303.209991  312.230011   
           ZTS     167.229996  167.229996  169.990005  161.380005  163.100006   

Price                   Volume  
Date       Ticker               
2016-05-05 A         1084000.0  
           AAL      11606100.0  
           AAPL    143562000.0  
           ABBV      8646200.0  
           ABT      16531000.0  
...                        ...  
2024-05-02 XYL       2784800.0  
           YUM       2425400.0  
           ZBH       2931700.0  
           ZBRA       507800.0  
           ZTS       5322500.0  

[988208 rows x 6 columns]


# 1.5.2 Make Data More Readable

In [25]:
df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-05 A        38.436123   40.959999   41.029999   40.509998   40.639999   
           AAL      31.531179   32.810001   33.520000   32.730000   33.279999   
           AAPL     21.438957   23.309999   23.517500   23.170000   23.500000   
           ABBV     44.179379   62.340000   62.400002   61.090000   61.410000   
           ABT      33.229504   38.500000   38.730000   38.099998   38.279999   
...                       ...         ...         ...         ...         ...   
2024-05-02 XYL     135.990005  135.990005  137.179993  132.020004  132.429993   
           YUM     135.070007  135.070007  136.610001  134.759995  135.500000   
           ZBH     118.459999  118.459999  123.959999  117.599998  121.989998   
           ZBRA    312.709991  312.709991  315.170013  303.209991  312.230011   
           ZTS     167.229996  167.229996  169.990005  161.380005  163.100006   

Price                   volume  
date       ticker               
2016-05-05 A         1084000.0  
           AAL      11606100.0  
           AAPL    143562000.0  
           ABBV      8646200.0  
           ABT      16531000.0  
...                        ...  
2024-05-02 XYL       2784800.0  
           YUM       2425400.0  
           ZBH       2931700.0  
           ZBRA       507800.0  
           ZTS       5322500.0  

[988208 rows x 6 columns]

In [36]:
df.to_csv("x.csv")

# 2. Calculate Different Technical Indicators per Stock

- RSI
- Bollinger Bands
- MACD
- ATR
- Garman-Klass Volatility
- Dollar Volume

In [32]:
def compute_atr(tickers_list):
    atr = pandas_ta.atr(high = tickers_list['high'],
                        low = tickers_list['low'],
                        close = tickers_list['close'],
                        length = 14)
    return atr.sub(atr.mean()).div(atr.std())   

In [33]:


def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())


In [38]:
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close = x, length= 20))

df['bb_low'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,0])

df['bb_mid'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,1])

df['bb_upper'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,2])

df ['atr'] = df.groupby (level = 1, group_keys = False).apply(compute_atr)

df['ema12'] = df['close'].ewm(span = 12, adjust = False).mean()

df['ema26'] = df['close'].ewm(span = 26, adjust = False).mean()

df['macd'] = df['ema12'] - df['ema26']

df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

In [39]:
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-05 A        38.436123   40.959999   41.029999   40.509998   40.639999   
           AAL      31.531179   32.810001   33.520000   32.730000   33.279999   
           AAPL     21.438957   23.309999   23.517500   23.170000   23.500000   
           ABBV     44.179379   62.340000   62.400002   61.090000   61.410000   
           ABT      33.229504   38.500000   38.730000   38.099998   38.279999   
...                       ...         ...         ...         ...         ...   
2024-05-02 XYL     135.990005  135.990005  137.179993  132.020004  132.429993   
           YUM     135.070007  135.070007  136.610001  134.759995  135.500000   
           ZBH     118.459999  118.459999  123.959999  117.599998  121.989998   
           ZBRA    312.709991  312.709991  315.170013  303.209991  312.230011   
           ZTS     167.229996  167.229996  169.990005  161.380005  163.100006   

Price                   volume        rsi    bb_low    bb_mid  bb_upper  \
date       ticker                                                         
2016-05-05 A         1084000.0        NaN       NaN       NaN       NaN   
           AAL      11606100.0        NaN       NaN       NaN       NaN   
           AAPL    143562000.0        NaN       NaN       NaN       NaN   
           ABBV      8646200.0        NaN       NaN       NaN       NaN   
           ABT      16531000.0        NaN       NaN       NaN       NaN   
...                        ...        ...       ...       ...       ...   
2024-05-02 XYL       2784800.0  68.550712  4.836793  4.871289  4.905784   
           YUM       2425400.0  43.375371  4.905459  4.938441  4.971423   
           ZBH       2931700.0  38.892780  4.763220  4.816631  4.870043   
           ZBRA       507800.0  62.286364  5.578997  5.674417  5.769836   
           ZTS       5322500.0  53.408617  4.971006  5.057631  5.144257   

Price                   atr       ema12       ema26       macd  \
date       ticker                                                
2016-05-05 A            NaN   40.959999   40.959999   0.000000   
           AAL          NaN   39.706153   40.356296  -0.650142   
           AAPL         NaN   37.183668   39.093607  -1.909939   
           ABBV         NaN   41.053873   40.815562   0.238311   
           ABT          NaN   40.660969   40.644039   0.016931   
...                     ...         ...         ...        ...   
2024-05-02 XYL     0.892935  118.974125  130.199460 -11.225335   
           YUM     0.682124  121.450414  130.560242  -9.109827   
           ZBH    -0.192443  120.990351  129.663927  -8.673577   
           ZBRA    0.513987  150.485680  143.222895   7.262785   
           ZTS     1.470820  153.061729  145.001199   8.060530   

Price              garman_klass_vol  
date       ticker                    
2016-05-05 A              -0.001120  
           AAL            -0.000841  
           AAPL           -0.003144  
           ABBV           -0.041668  
           ABT            -0.007599  
...                             ...  
2024-05-02 XYL             0.000463  
           YUM             0.000089  
           ZBH             0.001054  
           ZBRA            0.000747  
           ZTS             0.001109  

[988208 rows x 15 columns]

# 3. Aggregate on Monthly Level and Filter per Month the Most Liquid Stocks

# 4. Calculate Monthly Returns for Different Time-Horizons

# 5. Download Fama-French Factors; Calculate Rolling Factor Betas Per Stock

# 6.  Per Month, Create a K-means Clustering Model to group similar assets based on their Features

# 7. Per Month, Select Assets Based on the Cluster and Build a Portfolio Based on That

# 8. Visualize the Portfolio Returns and Compare That with the SP500 